# Machine Learning Modeling
For our purposes, we want to try a few different models to see which one works the best with our data. The first one we will try is Support Vector Machines. Before that, though, we'll need to split our data into training and testing.

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


data = pd.read_csv('data/moviedata_tokens.csv')

In [18]:
data.head()

,movie,character_name,line_num,line,tokens
0,American Psycho,Bateman,0,"we're sitting in pastels, this nouvelle north...","['we', ""'re"", 'sitting', 'in', ',', 'this', 'c..."
1,American Psycho,Bateman,1,you'll notice that my friends and i all look...,"['you', ""'ll"", 'notice', 'that', 'my', 'friend..."
2,American Psycho,Bateman,2,or can it be worn with a suit?,"['or', 'can', 'it', 'be', 'with', 'a', 'suit',..."
3,American Psycho,Bateman,3,with discreet pinstripes you should wear a sub...,"['with', 'you', 'should', 'wear', 'a', 'blue',..."
4,American Psycho,Bateman,4,van patten looks puffy. has he stopped working...,"['van', 'patten', 'looks', 'puffy', '.', 'has'..."


In [ ]:
# Splitting data into features (X) and labels (y)
'''
X = data['tokens']
y = data['character_name']

# Vectorizing the data
vectorizer = CountVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(X)

# Splitting into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=222)

# Creating the SVM model
svm = SVC(kernel='linear', C=1.0, random_state=222)

# Define cross-validation method and scoring metric
cv = KFold(n_splits = 5, shuffle=True, random_state=222)
scoring = 'f1_macro'
scores = []

# Loop over each fold of the cross-validation
for train_idx, val_idx in cv.split(X,y):
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = 

'''

In [19]:
# Splitting data into training and testing
X = data['tokens']
y = data['character_name']
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, test_size=0.2, random_state=222)

# Creating a pipeline
pipeline = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel='linear'))
])

# Creating K-Fold cross-validation model
kf = KFold(n_splits=10, shuffle=True, random_state=222)

# Compute the cross-validation scores for the pipeline
scores = cross_val_score(pipeline, X_train, y_train, cv=kf, n_jobs=-1)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


Accuracy: 0.42 (+/- 0.05)


let's try again without normalizing.

In [7]:
pipeline2 = Pipeline([
    ('vect', TfidfVectorizer(ngram_range=(1,2))),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(kernel='linear'))
])

# Creating K-Fold cross-validation model
kf = KFold(n_splits=10, shuffle=True, random_state=222)

# Compute the cross-validation scores for the pipeline
scores = cross_val_score(pipeline2, X_train, y_train, cv=kf)

print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.37 (+/- 0.03)


In [21]:
pipeline3 = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
    ('vect', TfidfTransformer()),
    ('clf', SVC())
])

# Define parameter grid to search over
param_grid = {
    'tfidf__max_features': [30000, 40000, 50000],
    'clf__C': [1, 10],
    'clf__kernel': ['linear']
}

# Creating K-Fold cross-validation model
kf = KFold(n_splits=10, shuffle=True, random_state=222)

# Define gGridSearchCV
grid_search = GridSearchCV(pipeline3, param_grid, cv=kf, n_jobs=-1)

# Fit the GridSearch CV object to the training data
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'clf__C': 10, 'clf__kernel': 'linear', 'tfidf__max_features': 50000}
Best score:  0.42539708741425475
